# Clustering

Clustering is an unsupervised learning method to group similar items together. Remember that data we sifted through earlier to predict how long it would take to read a Medium post? Let's take another look at that small data to see if we can determine anything interesting from it...

# 1. Converting data to usable form

The start of machine learning problems, as we know by now!

In [63]:
# import statements
import pandas as pd
import numpy as np
import nltk

nltk.download('punkt')

# start with data pre-processing (i.e. convert data.txt into something we can use)

# create class to hold data points
class MediumPost():
    def __init__(self, text):
        self.text = text
        self.tokens = nltk.word_tokenize(text)
        
    def get_text(self):
        return self.text
        
    def get_token_count(self):
        return len(self.tokens)
    
    def get_tokens(self):
        return self.tokens
    
    def get_bag_of_words(self, dictionary):
        counts = {}
        for token in self.tokens:
            token = token.lower() # let's make all tokens lowercase
            if token in counts:
                counts[token] += 1
            else:
                counts[token] = 1
        # let's normalize the counts to be percentages of the entire text (to account for some posts being longer than others)
        total = self.get_token_count()
        bow = [float(0)] * len(dictionary.keys())
        for token in counts:
            if token in dictionary:
                bow[dictionary[token]] = float(counts[token]) / float(total)
        return bow
        
posts = []
# iterate through the raw data file and separate out data points
with open("data.txt") as raw_data:
    for line in raw_data:
        if "TEXT:\t" in line:
            text = line.split("TEXT:\t")[1]
        elif "MINUTES:\t" in line:
            posts.append(MediumPost(text))
        else:
            text += line

# this is our dictionary that we will create - we will only use words in our bag of words that are within
dictionary = {}
for post in posts:
    for token in post.get_tokens():
        token = token.lower() # let's make all tokens lowercase
        if token in dictionary:
            dictionary[token] += 1
        else:
            dictionary[token] = 1
            
# now we have a dictionary of tokens that appear at least once in our data set
# let's update to only use the tokens that appear at least 3x in our data set
# let's also make sure we only take tokens that are at least 5 characters (many stop words of 4 or less)
# let's also account for some additional stop words
stop_words = ["which", "their", "about", "those", "because", "there", "would", "which", "could", "think"]
dictionary_tokens = []
# let's also
for token in dictionary:
    if dictionary[token] > 3 and len(token) > 4 and token not in stop_words:
        dictionary_tokens.append(token)
        
# and just for ease, let's create another dictionary associating each word with an index in our array
dictionary_tokens_indexes = {}
for i, token in enumerate(dictionary_tokens):
    dictionary_tokens_indexes[token] = i

# we want to store our data in a pandas dataframe
df = pd.DataFrame(columns = dictionary_tokens)

# iterate through the cleaner array of post objects and add data to our dataframe
for post in posts:
    df.loc[len(df)] = post.get_bag_of_words(dictionary_tokens_indexes)

# let's take a peak at our data
print(df.head())

[nltk_data] Downloading package punkt to /Users/akretch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
   financial     media  industry   falling     again    memory   ignored  \
0   0.000325  0.000325   0.00065  0.000976  0.001626  0.000325  0.000325   
1   0.001280  0.000640   0.00000  0.000000  0.000000  0.000000  0.001280   
2   0.000000  0.000000   0.00000  0.000000  0.000000  0.000000  0.000000   
3   0.000000  0.000000   0.00000  0.000000  0.000000  0.000000  0.000000   
4   0.000000  0.000000   0.00000  0.000000  0.000894  0.000000  0.000000   

      first  trouble     early   ...     protein  sweeteners  samish  hirsch  \
0  0.000976  0.00065  0.000650   ...         0.0         0.0     0.0     0.0   
1  0.000000  0.00000  0.000640   ...         0.0         0.0     0.0     0.0   
2  0.001493  0.00000  0.000000   ...         0.0         0.0     0.0     0.0   
3  0.001037  0.00000  0.001037   ...         0.0         0.0     0.0     0.0   
4  0.002235  0.00000  0

# 2. K-Means Clustering

K-Means clustering is a clustering algorithm that takes in a the number of desired clusters as a parameter (k), then randomly inserts k "centroid" points, and continues to toggle each of these points to the center of the points that are closest to it. Let's play around with this!

In [64]:
from sklearn.cluster import KMeans

# feel free to play around with the n_clusters parameter - we'll talk more about how to optimize this in a bit!
kmeans = KMeans(n_clusters=26)
predictions = kmeans.fit_predict(df)

print(predictions)

# print out any text you want as such
print(posts[0].get_text())
print(posts[4].get_text())

# try running this code block a few different times... do the predictions change?

[ 1  6  4  5  1  1 19 25  1 23  0 15 13 11  7  1  1  1  1 12 17  1  1 20
 14  1 22  1 18 16  3  2 21  1 24  8  1  9  6 10  1]
Back in 2008, the financial crisis kicked off a bloodbath for the media industry. Now, with the axe falling yet again at once-thriving outlets, a trip down memory lane.
I blithely ignored the first sign of trouble — a rare early morning phone call from a colleague as I walked to the subway. Had I checked my bank account? she asked. It seems the direct deposits hadn’t shown up, and the rest of the staff was freaking out. The previous night had been a late one, and frankly, I didn’t feel so hot. “Oh, come on. It’s not even 10 a.m.,” I told her wearily. “You guys need to chill out. I’m sure it’s fine.”
By this point, I’d been reassuring people for so long — patiently explaining to various staffers that what appeared from certain angles to be a falling sky was actually just an optical illusion — that I had myself convinced, too. But a few minutes later, after discov

# 3. Determining 'k'

The biggest challenge of K-Means clustering is providing the count of clusters you want. This is unsupervised learning after all, so we don't really know how many clusters we want! A common metric to score how strong of an association a cluster has to its members is calculating the silhouette score. The silhoette score is calculated by the formula (b - a) / max(a, b) where 'a' is the mean distance of a points in a cluster to each other and 'b' is the mean distance of points in a cluster to another cluster. This value can be between -1 and 1.

In [72]:
from sklearn.metrics import silhouette_score

possible_k_values = range(2, 40)
for k in possible_k_values:
    kmeans = KMeans(n_clusters=k)
    predictions = kmeans.fit_predict(df)
    score = silhouette_score(df, predictions)
    print("Score for k=%d: %.2f" % (k, score))

Score for k=2: 0.10
Score for k=3: 0.16
Score for k=4: 0.09
Score for k=5: 0.06
Score for k=6: 0.18
Score for k=7: 0.12
Score for k=8: 0.17
Score for k=9: 0.03
Score for k=10: 0.12
Score for k=11: 0.12
Score for k=12: 0.07
Score for k=13: 0.13
Score for k=14: 0.08
Score for k=15: 0.09
Score for k=16: 0.02
Score for k=17: 0.02
Score for k=18: 0.09
Score for k=19: 0.09
Score for k=20: 0.07
Score for k=21: 0.02
Score for k=22: 0.05
Score for k=23: 0.10
Score for k=24: 0.07
Score for k=25: 0.04
Score for k=26: 0.07
Score for k=27: 0.02
Score for k=28: 0.07
Score for k=29: 0.05
Score for k=30: 0.06
Score for k=31: 0.05
Score for k=32: 0.03
Score for k=33: 0.06
Score for k=34: 0.06
Score for k=35: 0.05
Score for k=36: 0.06
Score for k=37: 0.05
Score for k=38: 0.05
Score for k=39: 0.05


# 4. Interpretting the clusters

Which value of k shall we go with? Totally up to you! After you decide, let's run this code to interpret the tokens with the strongest weights to see if we can make sense out of this data.

In [73]:
import operator

kmeans = KMeans(n_clusters=6) # set your value of k for the value of n_clusters
predictions = kmeans.fit_predict(df)
centers = kmeans.cluster_centers_
print("Most significant tokens:")
for i, center in enumerate(centers):
    scores = {}
    for j, score in enumerate(center):
        if score > 0:
            scores[j] = score
    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)
    most_significant_tokens = ""
    for j, tup in enumerate(sorted_scores):
        key = tup[0]
        token = dictionary_tokens[key]
        if j == 0:
            most_significant_tokens += token
        elif j >= 10:
            break
        else:
            most_significant_tokens += ", %s" % token
    print("Cluster %d: %s" % (i, most_significant_tokens))

Most significant tokens:
Cluster 0: people, facebook, product, these, after, where, going, other, first, years
Cluster 1: amazon, power, company, antitrust, mitchell, retail, services, government, public, years
Cluster 2: adobe, invision, sketch, design, designers, cloud, software, collaboration, files, studio
Cluster 3: sexual, harassment, assault, women, survivors, trauma, percent, pressure, physical, found
Cluster 4: world, change, united, states, global, geopolitical, years, technology, people, continue
Cluster 5: fallout, product, quality, market, bethesda, fundamental, online, while, interplay, company


# 5. How can we improve upon this?